# Parsing the Parallax file

In [16]:
# imports
from pkg_resources import resource_filename
import numpy as np
import pdb

from astropy.table import Table

from pulsars import io as pio

## Load up the lines

In [2]:
p_file = resource_filename('pulsars', 'data/parallax/Parallaxes_aug2018.txt')

In [3]:
with open(p_file) as f:
    lines = [line.rstrip() for line in f]

In [4]:
lines[1]

'# Shami Chatterjee'

## Parse em

### This takes the first entry, which might not be the most accurate

In [5]:
all_obs = []
cnt = 0
while cnt < len(lines):
    if '!!psr' in lines[cnt]:
        tdict = {}
        cnt += 1
        tdict['JNAME'] = lines[cnt].split('=')[1].strip()
        cnt += 1
        tdict['BNAME'] = lines[cnt].split('=')[1].strip()
        cnt += 1
        tdict['PI'] = float(lines[cnt].split('=')[1].strip())    
        cnt += 1
        tdict['eplus'] = float(lines[cnt].split('=')[1].strip())
        cnt += 1
        tdict['eminus'] = float(lines[cnt].split('=')[1].strip())
        cnt += 1
        tdict['Ref'] = lines[cnt].split('=')[1].lstrip()
        cnt += 2
        tdict['Method'] = lines[cnt].split('=')[1].lstrip()    
        #
        all_obs.append(tdict.copy())
    else:
        cnt += 1

In [6]:
all_obs[0]

{'BNAME': '',
 'JNAME': 'J0030+0451',
 'Method': 'Timing',
 'PI': 3.3,
 'Ref': 'Lommen et al. (2006)',
 'eminus': -0.9,
 'eplus': 0.9}

## Table me

In [7]:
plx = Table(all_obs)
plx

BNAME,JNAME,Method,PI,Ref,eminus,eplus
str8,str13,str20,float64,str32,float64,float64
,J0030+0451,Timing,3.3,Lommen et al. (2006),-0.9,0.9
B0031-07,J0034-0721,VLBI (VLBA),0.93,Chatterjee et al. (2009),-0.07,0.08
B0037+56,J0040+5716,VLBI (VLBA),0.102,Deller et al. (2018),-0.025,0.051
,J0045-7319,Optical (Gaia DR2),0.04,Jennings et al. (2018),-0.06,0.06
B0052+51,J0055+5117,VLBI (VLBA),0.35,Deller et al. (2018),-0.06,0.06
B0059+65,J0102+6537,VLBI (VLBA),0.4,Deller et al. (2018),-0.04,0.04
,J0108-1431,VLBI (LBA),4.2,Deller et al. (2009),-1.4,1.4
B0105+65,J0108+6608,VLBI (VLBA),0.47,Deller et al. (2018),-0.03,0.04
B0136+57,J0139+5814,VLBI (VLBA),0.37,Chatterjee et al. (2009),-0.04,0.04


### Output

In [18]:
tbl_file = resource_filename('pulsars', 'data/parallax/Plx_aug2018.dat')

In [23]:
plx.write(tbl_file, overwrite=True, format='ascii.fixed_width')

## Match to DMs

In [10]:
DMs = pio.load_DM()
DMs[0:5]

Loading up DMs from /home/xavier/Projects/pulsars/pulsars/data/atnf_cat/DM_cat_v1.56.dat


ID,PSRJ,Pref,RAJ,RAf,RAref,DECJ,DECf,DECref,P0,P0f,P0ref,DM,DMf,DMref
int64,str12,str7,str16,int64,str7,str16,int64,str7,float64,int64,str7,float64,int64,str7
1,J0006+1834,cnt96,00:06:04.8,2,cn95,+18:34:59,4,cn95,0.69374767047,14,cn95,11.41,55,bkk+16
2,J0011+08,dsm+16,00:11:34,114,dsm+16,+08:10,8,dsm+16,2.55287,0,dsm+16,24.9,0,dsm+16
3,J0014+4746,dth78,00:14:17.75,4,hlk+04,+47:46:33.4,3,hlk+04,1.240699038946,11,hlk+04,30.405,13,bkk+16
4,J0023+0923,hrm+11,00:23:16.87910,3,mnf+16,+09:23:23.871,1,mnf+16,0.00305020310409707,3,abb+15,14.3283,5,abb+15
5,J0024-7204aa,phl+16,00:24:05.67,0,mam+06,-72:04:52.62,0,mam+06,0.00184,0,phl+16,24.971,7,phl+16


In [17]:
for row in plx:
    idx = np.where(DMs['PSRJ'] == row['JNAME'])[0]
    if len(idx) == 0:
        print(row)
    elif len(idx) > 1:
        pdb.set_trace()

BNAME   JNAME        Method     PI          Ref          eminus eplus
----- ---------- ------------- ---- -------------------- ------ -----
      J0720-3125 Optical (HST) 2.77 Kaplan et al. (2007)  -0.89  0.89
BNAME     JNAME       Method    PI          Ref          eminus eplus
----- ------------- ---------- ---- -------------------- ------ -----
      J0737-3039A/B VLBI (LBA) 0.87 Deller et al. (2009)  -0.14  0.14
BNAME   JNAME        Method     PI               Ref                eminus eplus
----- ---------- ------------- --- -------------------------------- ------ -----
      J1856-3754 Optical (HST) 6.2 van Kerkwijk &amp; Kaplan (2007)   -0.6   0.6
BNAME   JNAME          Method        PI           Ref           eminus eplus
----- ---------- ------------------ ---- ---------------------- ------ -----
      J2339-0533 Optical (Gaia DR2) 0.75 Jennings et al. (2018)  -0.25  0.25
